<a href="https://colab.research.google.com/github/Nilanshrajput/Video_Generation_Transformer/blob/master/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import os
from collections import OrderedDict
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import init
import functools
import torchvision.models as models

from __future__ import absolute_import, division, print_function, unicode_literals


from torchvision import models
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
from torchvision import transforms, utils
import matplotlib.pyplot as plt

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import re

In [0]:
import os
import re

# Suppose you are trying to load pre-trained resnet model in directory- models\resnet

os.environ['TORCH_HOME'] = 'D:\dev\Pytorch_Models\models\\resnet' #setting the environment variable


In [0]:

import json
import os
from collections import namedtuple
import zipfile

from PIL import Image


class CityscapeDataset(Dataset):
    """`Cityscapes <http://www.cityscapes-dataset.com/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory ``leftImg8bit``
            
        split (string, optional): The image split to use, ``train``, ``train_extra`` or ``val``
       
        transform (callable, optional): A function/transform that takes in a PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``  
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.

    Examples:

      
    """

    # Based on https://github.com/mcordts/cityscapesScripts
  

    def __init__(self, root=None, split='train_extra', transforms=None):
        if root is not None:
          self.images_dir = os.path.join(self.root, 'leftImg8bit', split)
        else:
          self.images_dir = os.path.join('leftImg8bit', split)
        self.split = split
        self.images = []
        self.root = root
        self.transforms = transforms
  
        #valid_modes = ("train", "train_extra", "val")
        
        for city in os.listdir(self.images_dir):
            img_dir = os.path.join(self.images_dir, city)
            images_city=[]
            for file_name in os.listdir(img_dir):
                
                images_city.append(os.path.join(img_dir, file_name))
            # re.split(\d+,input) splits by integer value
            images_city.sort( key= lambda text: int(re.split('(\d+)',text)[3]+re.split('(\d+)',text) [5] ))
            self.images+=images_city
        
    def print_dir(self):
      print(self.images)

    def im_path(self):
      return self.images            
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
             image 
        """

        image = Image.open(self.images[index]).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)

        return image


    def __len__(self):
        return len(self.images)



In [0]:
import json
import os
from collections import namedtuple
import zipfile

from PIL import Image


class CityscapeDataset(Dataset):
    """`Cityscapes <http://www.cityscapes-dataset.com/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory ``leftImg8bit``
            
        split (string, optional): The image split to use, ``train``, ``train_extra`` or ``val``
       
        transform (callable, optional): A function/transform that takes in a PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``  
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.

    Examples:

      
    """

    # Based on https://github.com/mcordts/cityscapesScripts
  

    def __init__(self, root=None, split='train_extra', transforms=None):
        if root is not None:
          self.images_dir = os.path.join(self.root, 'leftImg8bit', split)
        else:
          self.images_dir = os.path.join('leftImg8bit', split)
        self.split = split
        self.images = []
        self.root = root
        self.transforms = transforms
  
        #valid_modes = ("train", "train_extra", "val")
        
        for city in os.listdir(self.images_dir):
            img_dir = os.path.join(self.images_dir, city)
            images_city=[]
            for file_name in os.listdir(img_dir):
                
                images_city.append(os.path.join(img_dir, file_name))
            # re.split(\d+,input) splits by integer value
            images_city.sort( key= lambda text: int(re.split('(\d+)',text)[3]+re.split('(\d+)',text) [5] ))
            self.images+=images_city
        
    def print_dir(self):
      print(self.images)

    def im_path(self):
      return self.images            
    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
             image 
        """

        image = Image.open(self.images[index]).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)

        return image


    def __len__(self):
        return len(self.images)



In [0]:
scaler = transforms.Resize((224, 224))
to_tensor = transforms.ToTensor()

In [0]:
#not adding normalization as image looses textures and clrs for recostruction

composed_transforms=transforms.Compose([scaler,to_tensor])

In [0]:
transformed_dataset = CityscapeDataset(
                                           transforms=composed_transforms
                                           )

In [0]:
dataloader = DataLoader(transformed_dataset, batch_size=4,
                        sampler=torch.utils.data.SequentialSampler(transformed_dataset) , num_workers=4)

In [0]:
x=next(iter(dataloader))

In [0]:
class ResNet50Bottom(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Bottom, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-5])
        
    def forward(self, x):
        x = self.features(x)
        return x

In [0]:
res50_model = models.resnet50(pretrained=True)
res50_conv2 = ResNet50Bottom(res50_model).cuda()

outputs = res50_conv2(x.to('cuda'))
outputs.data.shape

In [0]:
class FeatureChanger(nn.Module):
    def __init__(self,in_channels, out_channels):
        super().__init__()
        
        self.kernel_size = 1
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.conv2d_0 = torch.nn.Conv2d(in_channels, out_channels*2, kernel_size, stride=1)
        self.con2d_1 = torch.nn.Conv2d(out_channels*2, out_channels, kernel_size, stride=1)
        
    def forward(self, batch):
        x = self.conv2d_0(batch)
        x = self.con2d_1(x)
        return x

In [0]:
class MaskedSelfAttention(nn.Module):

  def __init__(self,heads, d_model):
    super().__init__()

    self.d_model = d_model
    self.heads = heads


  def _generate_square_subsequent_mask(self, sz):
    #sz= batch size
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

  def mult_4d(self, batch, q_w, bs):
    layers = batch.shape[1]
    dim1 = batch.shape[2]
    dim2 = batch.shape[3]
    output = torch.bmm(batch.view(bs*layers,dim1,dim2), q_w.view(bs*layers,dim1,dim2))
    return output.view(bs,layers,dim1,dim2)

  def forward(self):

    #score
    attn_output_weights = mult_4d(q, k)

In [0]:

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_3d = torch.nn.Parameter(torch.empty(bs,nu_feat,dim1,dim2,requires_grad=True))
        self.k_3d = torch.nn.Parameter(torch.empty(bs,nu_feat,dim1,dim2,requires_grad=True))
        self.v_3d = torch.nn.Parameter(torch.empty(bs,nu_feat,dim1,dim2,requires_grad=True))
   

        nn.init.kaiming_normal_(q_3d, mode='fan_in')
        nn.init.kaiming_normal_(k_3d, mode='fan_in')
        nn.init.kaiming_normal_(v_3d, mode='fan_in')

        self.attention_layer= MaskedSelfAttention()
        
        
        #self.q_linear = nn.Linear(d_model, d_model)
        #self.v_linear = nn.Linear(d_model, d_model)
        #self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def mult_4d(self, batch, q_w, bs):
      layers = batch.shape[1]
      dim1 = batch.shape[2]
      dim2 = batch.shape[3]
      output = torch.bmm(batch.view(bs*layers,dim1,dim2), q_w.view(bs*layers,dim1,dim2))
      return output.view(bs,layers,dim1,dim2)

    def forward(self, batch, mask=None):
        
        bs = q.size(0)
        features_changer = FeatureChanger(batch.shape[1],batch.shape[1]/self.h*2)
        batch = feature_changer(batch)

        q = mult_4d(batch,self.q_3d,bs)
        k = mult_4d(batch,self.k_3d,bs)
        v = mult_4d(batch,self.v_3d,bs)
        #divide feature space in no of heads each head works on certain features only
        #returns tuple of size=heads

        batch_chunks= torch.chunk(batch, heads, dim=1)
        #converting tuple to tensor
        batch_chunks = torch.stack(batch_chunks, dim =0)

        attention_out= attention_layer(batch_chunks, q, k, v)
        
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
# calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [0]:
w = torch.empty(500,300)
a=  torch.empty(500,300)
a= nn.init.kaiming_uniform_(a, mode='fan_in')



nn.init.kaiming_uniform_(w, mode='fan_in')
(a*w).argmax()

tensor(29802)

In [0]:
w = torch.rand(32,256,20,20)
w.argmax()

tensor(2207442)

In [0]:
a=torch.chunk(w,8,dim=1)

In [0]:
c=torch.reshape(w,(8,32,32,20,20))

In [0]:
d=torch.stack(a,dim=0)

In [0]:
torch.equal(c,d)

False

In [0]:
torch.all(torch.lt(torch.abs(torch.add(c, -d)), 1e-1))

tensor(False)

In [0]:
sz=4
mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
print(mask)
mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

tensor([[ True, False, False, False],
        [ True,  True, False, False],
        [ True,  True,  True, False],
        [ True,  True,  True,  True]])


In [0]:
mask

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [0]:
mask = (torch.triu(torch.ones(sz, sz)) == 1)

In [0]:
def mult_4d( batch, q_w, bs):
  layers = batch.shape[1]
  dim1 = batch.shape[2]
  dim2 = batch.shape[3]
  output = torch.bmm(batch.view(bs*layers,dim1,dim2), q_w.view(bs*layers,dim1,dim2))
  return output.view(bs,layers,dim1,dim2)

In [0]:
w = torch.rand(4,2,2,2)
a = torch.rand(4,2,2,2)
#torch.bmm(w,a).shape

In [0]:
def generate_square_subsequent_mask(sz):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

mask= generate_square_subsequent_mask(4)
mask

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [0]:
mask.unsqueeze(0).unsqueeze(0)+a

RuntimeError: ignored

In [0]:
mask.unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 4, 4])

In [0]:
l=nn.Softmax2d()#.shape

In [0]:
c= l(mult_4d(w,a,w.shape[0]))

In [0]:
c.shape

torch.Size([4, 2, 2, 2])

In [0]:
l=nn.Softmax2d()#.shape

In [0]:
c=l(mult_4d(w,a,w.shape[0]))